# Importações

In [1]:
import random
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
batch_size = 32
train = (10000 // batch_size) * batch_size
val = (3000 // batch_size) * batch_size
train, val

(9984, 2976)

**Explicação do código:**

As linhas de código definem o tamanho do lote de dados como 32 e calculam o número total de amostras usadas para treinamento e validação, garantindo que sejam múltiplos exatos desse valor. O número de amostras de treinamento (*train*) é obtido dividindo 10000 por 32, pegando apenas a parte inteira do resultado e multiplicando novamente por 32, o que dá 9984 amostras. O mesmo processo é feito para as amostras de validação (*val*), dividindo 3000 por 32, resultando em 2976 amostras.

Isso garante que todas as amostras sejam usadas em lotes completos, evitando problemas ao dividir os dados durante o treinamento e a validação.

# Implementação do Zero

In [3]:
class SyntheticRegressionData:

    def __init__(self, w, b, noise=0.01, num_train=train, num_val=val, batch_size=batch_size):
        self.w = w
        self.b = b
        self.noise = noise
        self.num_train = num_train
        self.num_val = num_val
        self.batch_size = batch_size

        n = num_train + num_val
        self.X = torch.randn(n, len(w))
        noise = torch.randn(n, 1) * noise
        self.y = torch.matmul(self.X, w.reshape((-1, 1))) + b + noise

    def get_dataloader(self, train):
        indices = list(range(0, self.num_train)) if train else list(range(self.num_train, self.num_train + self.num_val))
        random.shuffle(indices)
        for i in range(0, len(indices), self.batch_size):
            batch_indices = torch.tensor(indices[i: i + self.batch_size])
            yield self.X[batch_indices], self.y[batch_indices]

**Explicação do código:**

Este código cria uma classe *SyntheticRegressionData* que gera dados sintéticos para um problema de regressão linear. Eu uso pesos *w* e um viés *b* para criar uma relação linear entre as variáveis de entrada *X* e a variável de saída *y*, com um ruído adicionado para simular dados reais. O construtor *(__init__)* inicializa os dados, criando *X* como uma matriz de valores aleatórios e *y* como uma combinação linear de *X*, *w* e *b*, com ruído adicionado.

Eu também divido os dados em conjuntos de treino e validação, com tamanhos definidos por *num_train* e *num_val*. O método *get_dataloader* permite iterar sobre os dados em lotes (*batch_size*), embaralhando os *índices* para treino e retornando os pares *(X, y)* correspondentes. Esse código é útil para simular e testar modelos de regressão linear em um ambiente controlado.

In [4]:
class LinearRegressionScratch:

    def __init__(self, num_inputs, lr, sigma=0.01):
        self.lr = lr
        self.w = torch.normal(0, sigma, (num_inputs, 1), requires_grad=True)
        self.b = torch.zeros(1, requires_grad=True)

    def forward(self, X):
        return torch.matmul(X, self.w) + self.b

    def loss(self, y_hat, y):
        return ((y_hat - y) ** 2 / 2).mean()

    def parameters(self):
        return [self.w, self.b]

**Explicação do código:**

Este código implementa uma regressão linear do zero usando PyTorch. Eu inicializo os pesos w com
valores aleatórios de uma distribuição normal e o viés b com zero, ambos com *requires_grad=True*
para calcular gradientes. No método *forward*, calculo a previsão *y_hat* usando a fórmula $y_{hat} = X \cdot$
w + b, onde *torch.matmul(X, self.w)* faz a multiplicação de matrizes. 

A função *loss* calcula o erro quadrático médio dividido por 2: $Loss = \frac{1}{2} \cdot (y_{hat} - y)^2$, e retorno a média do erro. Por fim, o método *parameters* retorna os parâmetros w e b para atualização durante o treinamento.

In [5]:
class SGD:

    def __init__(self, params, lr):
        self.params = params
        self.lr = lr

    def step(self):
        for param in self.params:
            param.data = param.data - self.lr * param.grad

    def zero_grad(self):
        for param in self.params:
            if param.grad is not None:
                param.grad.zero_() 

**Explicação do código:**

Essa classe implementa o gradiente descendente estocástico (SGD). Na função *step*, eu atualizo os
parâmetros do modelo usando a fórmula $param.data = param.data - lr \cdot param.grad$, onde *lr* é a taxa de
aprendizagem e *param.grad* é o gradiente do parâmetro. Isso move os parâmetros na direção que
minimiza a função de perda.

Já na função *zero_grad*, eu zero os gradientes dos parâmetros após cada
atualização, para evitar que eles interfiram no próximo passo de treinamento. Essas duas funções são
essenciais para o treinamento de modelos com gradiente descendente!

In [6]:
class Trainer:

    def __init__(self, max_epochs, gradient_clip_val=0, decay_rate=0.000001):
        self.max_epochs = max_epochs
        self.gradient_clip_val = gradient_clip_val
        self.decay_rate = decay_rate
        self.step_count = 0

    def prepare_data(self, data):
        self.train_dataloader = list(data.get_dataloader(train=True))
        self.val_dataloader = list(data.get_dataloader(train=False))

    def prepare_model(self, model):
        self.model = model

    def fit(self, model, data):
        self.prepare_data(data)
        self.prepare_model(model)
        self.optim = SGD(model.parameters(), model.lr)
        for epoch in tqdm(range(self.max_epochs), desc='Training'):
            self.fit_epoch()

    def fit_epoch(self):
        train_loss = 0.0
        num_batches = 0

        for X, y in self.train_dataloader:
            y_hat = self.model.forward(X)
            loss = self.model.loss(y_hat, y)
            loss.backward()
            self.step_count += 1
            self.optim.lr *= (1 / (1 + self.decay_rate * self.step_count))
            self.optim.step()
            self.optim.zero_grad()
            
            train_loss += loss.item()
            num_batches += 1

        train_loss /= num_batches
        print(f'Époch: {self.step_count}, Train Loss: {train_loss:.4f}, Learning Rate: {self.optim.lr:.6f}')
        
        if self.val_dataloader:
            val_loss = 0.0
            num_batches = 0
            
            for X, y in self.val_dataloader:
                with torch.no_grad():
                    y_hat = self.model.forward(X)
                    loss = self.model.loss(y_hat, y)
                    val_loss += loss.item()
                    num_batches += 1

            val_loss /= num_batches
            print(f'Époch: {self.step_count}, Validation Loss: {val_loss:.4f}')

**Explicação do código:**

O *Trainer* é responsável por treinar um modelo de aprendizado de máquina
utilizando gradiente descendente. No início, os valores de *max_epochs*,
*gradient_clip_val* e *decay_rate* são definidos por padrão, e *step_count* é inicializado em zero.

A função *prepare_data* carrega os dados de treinamento e validação usando
*data.get_dataloader(train=True)* para o conjunto de treinamento e
*data.get_dataloader(train=False)* para o conjunto de validação. A função
*prepare_model* recebe o modelo que será treinado.

A função *fit* inicia o treinamento chamando *prepare_data* e *prepare_model*. Em
seguida, cria o otimizador *SGD* com os parâmetros do modelo e a taxa de aprendizado
inicial. O treinamento ocorre em um loop de epochs, onde *fit_epoch* é chamado
repetidamente.

Dentro de *fit_epoch*, *train_loss* e *num_batches* são inicializados em zero. Para cada lote
de dados de entrada $X$ e rótulos $y$ no conjunto de treinamento, a previsão do modelo
é calculada como $\hat{y} = Xw + b$. A perda é então obtida com a fórmula $loss = \frac{1}{2}(\hat{y} -
y)^2$.

O método *loss.backward()* computa os gradientes e incrementa *step_count*. A taxa de
aprendizado é atualizada de acordo com $lr = \frac{lr}{1+decay\_rate \cdot step\_count}$, reduzindo
gradativamente. Os parâmetros do modelo são ajustados usando $param = param - lr \cdot param.grad$, e os gradientes são zerados para evitar acumulação.

A perda média do treinamento é calculada e exibida na tela junto com a taxa de
aprendizado. Se houver um conjunto de validação, a perda de validação é calculada sem
atualizar os parâmetros, usando $loss = \frac{1}{2}(\hat{y} - y)^2$, e o resultado também é exibido.

In [7]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2

**Explicação do código:**

O *true_w* recebe os valores dos pesos e o *true_b* recebe o valor do viés (bias)

In [8]:
data = SyntheticRegressionData(w=true_w, b=true_b)

In [9]:
model = LinearRegressionScratch(num_inputs=2, lr = 0.01)

In [10]:
trainer = Trainer(max_epochs=4)

**Explicação do código:**

A primeira linha cria um conjunto de dados sintético para regressão, onde os pesos verdadeiros dos dados são definidos por *true_w* e o viés verdadeiro por *true_b*.

A segunda linha instancia um modelo de regressão linear do zero, especificando que ele terá duas entradas e uma taxa de aprendizado inicial de 0.01. 

A terceira linha cria um treinador que irá treinar o modelo por 4 épocas.

In [11]:
trainer.fit(model, data)

Training: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 17.56it/s]

Époch: 312, Train Loss: 2.6760, Learning Rate: 0.009523
Époch: 312, Validation Loss: 0.0307
Époch: 624, Train Loss: 0.0052, Learning Rate: 0.008229
Époch: 624, Validation Loss: 0.0001
Époch: 936, Train Loss: 0.0001, Learning Rate: 0.006451
Époch: 936, Validation Loss: 0.0001
Époch: 1248, Train Loss: 0.0001, Learning Rate: 0.004588
Époch: 1248, Validation Loss: 0.0000


**Explicação do treinamento:**

A primeira linha chama o método de treinamento, iniciando o processo de ajuste do modelo aos dados. O treinamento ocorre por quatro épocas, conforme especificado anteriormente. Durante esse processo, a barra de progresso mostra que todas as épocas foram concluídas rapidamente. 

A cada conjunto de 312 iterações, são exibidos os valores de perda para os dados de treino e validação, além da taxa de aprendizado ajustada. No início, a perda no treino é alta, mas reduz rapidamente à medida que o modelo aprende, enquanto a perda na validação também diminui e se mantém muito baixa. A taxa de aprendizado diminui gradualmente ao longo das iterações, o que ajuda o modelo a ajustar os pesos com mais precisão à medida que se aproxima de um bom ajuste.

In [12]:
with torch.no_grad():
    print(f'Erro na estimação de w: {true_w - model.w.reshape(true_w.shape)}')
    print(f'Erro na estimação de b: {true_b - model.b}')

Erro na estimação de w: tensor([-6.2466e-05, -2.2578e-04])
Erro na estimação de b: tensor([0.0002])


**Conclusão do estudo de caso:**

O código desenvolvido implementa uma rede neural para regressão linear do zero, adotando uma abordagem matemática estruturada. Os dados sintéticos foram gerados com pesos e viés conhecidos, permitindo avaliar a precisão do modelo. O treinamento ocorreu de forma eficiente, com uma rápida convergência da função de perda, enquanto a validação demonstrou que o modelo generaliza bem.

A otimização com SGD e decaimento da taxa de aprendizado contribuiu para um ajuste preciso dos parâmetros. O resultado final confirma o sucesso da abordagem, com erros mínimos na estimativa dos pesos e do viés, demonstrando que o modelo aprendeu corretamente os valores esperados. Esse estudo de caso reforça a importância do entendimento matemático na construção de redes neurais do zero.